In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import librosa
import os
from tqdm import tqdm

In [15]:
!pip install resampy

In [16]:
# Define emotions
emotions = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}

In [17]:
def extract_features(file_path):
    audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    chroma = librosa.feature.chroma_stft(y=audio, sr=sample_rate)
    mel = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_mels=128)
    contrast = librosa.feature.spectral_contrast(y=audio, sr=sample_rate)
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(audio), sr=sample_rate)
    
    features = np.concatenate((
        np.mean(mfccs.T, axis=0), np.std(mfccs.T, axis=0),
        np.mean(chroma.T, axis=0), np.std(chroma.T, axis=0),
        np.mean(mel.T, axis=0), np.std(mel.T, axis=0),
        np.mean(contrast.T, axis=0), np.std(contrast.T, axis=0),
        np.mean(tonnetz.T, axis=0), np.std(tonnetz.T, axis=0)
    ))
    
    return features

In [18]:
# Load and extract features
features = []
labels = []
main_dir = '/kaggle/input/ravdess-emotional-speech-audio'  

In [19]:
# Check if directory exists
if not os.path.exists(main_dir):
    print(f"Error: The directory {main_dir} does not exist.")
    exit()

# Get total number of .wav files for progress bar
total_files = sum([len(files) for r, d, files in os.walk(main_dir) if any(file.lower().endswith('.wav') for file in files)])

if total_files == 0:
    print(f"Error: No '.wav' files found in the directory {main_dir}.")
    exit()

In [ ]:
# Create progress bar
pbar = tqdm(total=total_files, desc="Extracting features")

for root, dirs, files in os.walk(main_dir):
    print(f"Current directory: {root}")  # Debugging: See which directories are being traversed
    for file in files:
        if file.lower().endswith('.wav'):  # Handle both '.wav' and '.WAV' extensions
            file_path = os.path.join(root, file)
            try:
                print(f"Processing file: {file}")  # Debugging: See which files are being processed
                feature = extract_features(file_path)
                features.append(feature)
                emotion_code = file.split('-')[2]
                label = emotions[emotion_code]
                labels.append(label)
            except Exception as e:
                print(f"Error processing file {file}: {str(e)}")
            pbar.update(1)  # Update progress bar

pbar.close()  # Close progress bar

features = np.array(features)
labels = np.array(labels)

print(f"Total files processed: {len(features)}")

# Check if any files were processed
if len(features) == 0:
    print("Error: No features were extracted. Please check the file structure or contents of the directory.")
    exit()

Extracting features:   0%|          | 0/2880 [00:00<?, ?it/s]

Current directory: /kaggle/input/ravdess-emotional-speech-audio
Current directory: /kaggle/input/ravdess-emotional-speech-audio/Actor_02
Processing file: 03-01-08-01-01-01-02.wav


Extracting features:   0%|          | 1/2880 [00:00<21:42,  2.21it/s]

Processing file: 03-01-01-01-01-01-02.wav


Extracting features:   0%|          | 2/2880 [00:00<22:45,  2.11it/s]

...

Processing file: 03-01-04-02-02-01-19.wav


Extracting features:  35%|███▌      | 1015/2880 [09:11<17:45,  1.75it/s]

Processing file: 03-01-03-01-01-01-19.wav


Extracting features:  35%|███▌      | 1016/2880 [09:11<18:20,  1.69it/s]

Processing file: 03-01-06-02-01-01-19.wav


In [ ]:
# Analyze dataset
unique_labels, counts = np.unique(labels, return_counts=True)
print("\nDataset composition:")
for label, count in zip(unique_labels, counts):
    print(f"{label}: {count}")

In [ ]:
# Balance dataset
min_count = np.min(counts)
if min_count == 0:
    print("\nWarning: One or more emotion categories are empty. Skipping dataset balancing.")
    balanced_features = features
    balanced_labels = labels
else:
    print(f"\nBalancing dataset to {min_count} samples per category")
    balanced_features = []
    balanced_labels = []
    for label in unique_labels:
        indices = np.where(labels == label)[0]
        selected_indices = np.random.choice(indices, min_count, replace=False)
        balanced_features.extend(features[selected_indices])
        balanced_labels.extend(labels[selected_indices])

features = np.array(balanced_features)
labels = np.array(balanced_labels)

print(f"Total files after balancing: {len(features)}")

# Check if we have enough data to proceed
if len(features) < 2:
    print("Error: Not enough data to train the model. Please check your dataset.")
    exit()

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.20, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Encode labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

# Build and train MLP model
mlp = MLPClassifier(max_iter=1000)
mlp.fit(X_train_scaled, y_train_encoded)

In [ ]:
# Fine-tune hyperparameters
param_dist = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

print("Starting RandomizedSearchCV. This may take a while...")
random_search = RandomizedSearchCV(mlp, param_distributions=param_dist, n_iter=20, cv=5, n_jobs=-1, verbose=2)
random_search.fit(X_train_scaled, y_train_encoded)

best_mlp = random_search.best_estimator_
print(f"Best parameters: {random_search.best_params_}")

In [ ]:
# Make predictions
y_pred = best_mlp.predict(X_test_scaled)

# Analyze metrics
print("\nClassification Report:")
print(classification_report(y_test_encoded, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test_encoded, y_pred))

# K-Fold Cross-Validation
print("\nPerforming K-Fold Cross-Validation...")
cv_scores = cross_val_score(best_mlp, X_train_scaled, y_train_encoded, cv=5, n_jobs=-1)
print(f"Cross-validation scores: {cv_scores}")
print(f"Mean CV score: {np.mean(cv_scores)}")

# Comparative study with SVM and KNN
print("\nTraining SVM...")
svm = SVC()
svm.fit(X_train_scaled, y_train_encoded)

print("Training KNN...")
knn = KNeighborsClassifier()
knn.fit(X_train_scaled, y_train_encoded)

svm_pred = svm.predict(X_test_scaled)
knn_pred = knn.predict(X_test_scaled)

print("\nSVM Performance:")
print(classification_report(y_test_encoded, svm_pred))

print("\nKNN Performance:")
print(classification_report(y_test_encoded, knn_pred))

In [ ]:
# Visualize features
def plot_features(features, feature_name, num_features):
    plt.figure(figsize=(10, 4))
    plt.imshow(features[:num_features].T, aspect='auto', interpolation='nearest')
    plt.title(f'{feature_name} Features')
    plt.ylabel('Feature')
    plt.xlabel('Frame')
    plt.colorbar()
    plt.tight_layout()
    plt.show()

# Example usage (adjust slice indices based on your feature vector structure)
plot_features(features[:, :13], 'MFCC', 13)
plot_features(features[:, 13:25], 'Chromagram', 12)
plot_features(features[:, 25:125], 'Mel-scaled Spectrogram', 100)
plot_features(features[:, 125:132], 'Spectral Contrast', 7)
plot_features(features[:, 132:], 'Tonnetz', 6)

# Visualize confusion matrix
def plot_confusion_matrix(y_true, y_pred, classes):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.tight_layout()
    plt.show()

# Plot confusion matrix for MLP
plot_confusion_matrix(y_test_encoded, y_pred, le.classes_)